In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("C:/Users/biswa/Downloads/Datasets-2025-08-08/news_events_final.csv")
df.head(5)

,event_id,event_type,summary,category,found_at,confidence,article_sentence,human_approved,planning,effective_date,location,company_id,company_name,company_domain,company_ticker,article_id,article_title,article_url,article_published_at,article_body
0,0020f127-3470-4cce-8989-1c79f45da217,news_event,Unipart Manufacturing Group recognized as Tran...,recognized_as,2022-07-10 20:00:00,0.8759,In addition to being named the safest organisa...,False,False,2022-07-10 20:00:00,United Kingdom,000bd323-1bf8-5c7a-9941-e6c155c29d10,Unipart Manufacturing Group,unipart.com,Unknown,d172abc1-3755-4cef-946e-7de944806e7d,Unipart ranked the safest organisation in the ...,https://www.unipart.com/unipart-ranked-the-saf...,2022-07-10 20:00:00,Unipart has been further recognised for its un...
1,009be1ff-6cfb-4e9f-a415-69baf71f47f3,news_event,OOS International received award two safety aw...,receives_award,2019-12-19 10:45:17,0.9497,Since then OOS International has been an activ...,False,False,2018-01-01 00:00:00,Unknown,000ff896-4292-5b15-9c81-8bf4d76c10d7,OOS International,oosinternational.com,Unknown,58c0d5fd-068d-4bab-8ac4-47e19bbdf091,OOS International Award for completing another...,https://www.oosinternational.com/oos-news/awar...,2019-12-19 10:45:17,"On behalf of CIM OOS Tiradentes, Mr Jose Marci..."
2,01444124-7375-4f03-8879-eb8200b31504,news_event,NWN Corporation received award Global Winner f...,receives_award,2022-07-12 20:00:00,0.6887,"As a result, with nearly 400 nominees from ove...",False,False,2022-06-28 00:00:00,Unknown,000d8a9c-882c-57f2-8b4c-2afc786d0fa1,NWN Corporation,nwncarousel.com,Unknown,ef330a38-8624-41c1-8b75-d1b96e7dbd45,Skyrocketing Employee Engagement with Microsof...,https://nwncarousel.com/blog/skyrocketing-empl...,2022-07-12 22:00:00,More than 270 million people rely on Teams for...
3,031a304c-29ca-415e-a815-e9c915896540,news_event,Grape Solutions Plc. is developing Mobiliti ap...,is_developing,2023-04-02 22:00:00,0.5987,MVM Mobiliti and Grape Solutions have been wor...,False,False,2018-01-01 00:00:00,Hungary,0008b75f-9d15-54ae-b70a-52301945e397,Grape Solutions Plc.,grape.solutions,Unknown,0525807d-6ff6-44a0-9c36-8be3afceba5b,Hungarian development makes electric vehicle c...,https://www.iqfin.net/technology/hungarian-dev...,2023-04-02 22:00:00,Grape Solutions and MVM Mobiliti partner with ...
4,037783ca-f3f7-4782-8a81-df3cae1ac936,news_event,"NWN Corporation launched two new kits, At-Home...",launches,2022-04-13 01:02:36,0.7180,"NWN Carousel, the leading integrated cloud com...",False,False,2022-04-13 00:00:00,Unknown,000d8a9c-882c-57f2-8b4c-2afc786d0fa1,NWN Corporation,nwncarousel.com,Unknown,16061c55-111d-496a-9e3e-837dddc3454b,NWN Carousel Launches Two Hybrid Work Solution...,https://www.businesswire.com/news/home/2022041...,2022-04-13 03:00:00,"NWN Carousel, the leading integrated cloud com..."


### SQL CONNECTION

In [3]:
from sqlalchemy import create_engine

# Replace with your credentials
db_user = 'root'
db_password = '290506'
db_host = 'localhost'
db_name = 'news_db'

# Create engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')


In [16]:
# Extract unique companies
companies_df = df[['company_id', 'company_name', 'company_domain', 'company_ticker']].drop_duplicates()

# Extract unique articles
articles_df = df[['article_id', 'article_title', 'article_url', 'article_published_at', 'article_body']].drop_duplicates()

# Extract events
events_df = df[['event_id', 'company_id', 'article_id', 'category', 'summary', 
                'found_at', 'effective_date', 'confidence', 'human_approved', 
                'planning','location']]


In [23]:
import pandas as pd
from sqlalchemy import create_engine

# -----------------------------
# Step 1: Connect to MySQL
# -----------------------------
user = 'root'
password = '290506'
host = 'localhost'
port = '3306'
database = 'news_db'

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")


In [25]:


# -----------------------------
# Step 2: Prepare DataFrames
# -----------------------------
# Example: df_companies, df_articles, df_events are your cleaned DataFrames

#2a. Ensure datetime columns are correct
if 'article_published_at' in articles_df.columns:
    articles_df['article_published_at'] = pd.to_datetime(articles_df['article_published_at'], errors='coerce')

if 'found_at' in events_df.columns:
    events_df['found_at'] = pd.to_datetime(events_df['found_at'], errors='coerce')

if 'effective_date' in events_df.columns:
    events_df['effective_date'] = pd.to_datetime(events_df['effective_date'], errors='coerce')

# 2b. Ensure boolean columns are 0/1
for col in ['human_approved', 'planning']:
    if col in events_df.columns:
        events_df[col] = events_df[col].fillna(0).astype(int)

# 2c. Fill missing string columns with empty string
string_columns = ['company_name', 'company_domain', 'company_ticker', 
                  'article_title', 'article_url', 'article_author', 
                  'location', 'summary', 'article_sentence', 'category', 'event_type']
for col in string_columns:
    if col in events_df.columns:
        events_df[col] = events_df[col].fillna('')
    if col in articles_df.columns:
        articles_df[col] = articles_df[col].fillna('')
    if col in companies_df.columns:
        companies_df[col] = companies_df[col].fillna('')

C:\Users\biswa\AppData\Local\Temp\ipykernel_21484\4148060421.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_df['found_at'] = pd.to_datetime(events_df['found_at'], errors='coerce')
C:\Users\biswa\AppData\Local\Temp\ipykernel_21484\4148060421.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_df['effective_date'] = pd.to_datetime(events_df['effective_date'], errors='coerce')
C:\Users\biswa\AppData\Local\Temp\ipykernel_21484\4148060421.py:19: SettingWithCopyWarning: 
A value is trying to 

In [26]:
 #Insert companies
companies_df.to_sql('companies', con=engine, if_exists='append', index=False)


93622

In [27]:
# Insert articles
articles_df.to_sql('articles', con=engine, if_exists='append', index=False)


563433

In [28]:



# -----------------------------
# Step 3: Insert into MySQL
# -----------------------------


# Insert events
events_df.to_sql('events', con=engine, if_exists='append', index=False)

print("✅ All data imported successfully!")


✅ All data imported successfully!


In [29]:

# Example: check first 5 rows in 'events' table
df_check = pd.read_sql("SELECT * FROM events LIMIT 5;", engine)
print(df_check)


                               event_id event_type  \
0  00000514-1a87-4c23-a997-8bb070e47d28       None   
1  0000079a-d309-4192-8bf9-48884aafab39       None   
2  000008d3-b971-474b-bdf0-fe5626ac8f73       None   
3  00001347-ab48-464a-87c7-34261c8c674d       None   
4  00002c53-81ef-45ca-9569-0308c90913b6       None   

                                             summary             category  \
0  Universitas Gadjah Mada attended event Dies Na...        attends_event   
1     TCW Group expands offices to Hong Kong, China.   expands_offices_to   
2             Taslittleathletics partners with Rawc.        partners_with   
3           Epic Games launched SMG on Jul 24th '18.             launches   
4  Btgpactualdigital invests into assets: stake i...  invests_into_assets   

             found_at  confidence article_sentence  human_approved  planning  \
0 2024-12-28 06:44:29      0.4957             None               0         0   
1 2013-06-16 18:01:28      0.0470             None  

In [6]:
# -----------------------------------------------------
# 2️⃣ Compute Data Quality Metrics
# -----------------------------------------------------

# --- Completeness: ratio of non-null to total values ---
completeness = 1 - (df.isnull().sum().sum() / (df.shape[0] * df.shape[1]))

# --- Uniqueness: based on event_id (or company_id if duplicates are across events) ---
uniqueness = df['event_id'].nunique() / len(df)

# --- Accuracy: proxy metric — assume 'confidence' column indicates reliability ---
accuracy = df['confidence'].mean() / 1.0  # normalized since confidence ≤ 1

# --- Timeliness: percentage of recent events (within last 1 year from latest found_at) ---
df['found_at'] = pd.to_datetime(df['found_at'], errors='coerce')
max_date = df['found_at'].max()
timeliness = len(df[df['found_at'] > (max_date - pd.Timedelta(days=365))]) / len(df)

# --- Validity: proportion of valid URLs ---
valid_urls = df['article_url'].str.startswith(("http://", "https://")).sum()
validity = valid_urls / len(df)


In [7]:
# 3️⃣ Prepare metrics as DataFrame
# -----------------------------------------------------
dq_metrics = pd.DataFrame({
    'metric_name': ['Completeness', 'Uniqueness', 'Accuracy', 'Timeliness', 'Validity'],
    'metric_value': [completeness, uniqueness, accuracy, timeliness, validity],
    'notes': [
        'Ratio of non-null fields across all records',
        'Distinct event_id count divided by total events',
        'Average confidence score as proxy for data accuracy',
        'Events updated or created within last 1 year',
        'Share of valid article URLs'
    ]
})

print("📊 Computed Data Quality Metrics:")
print(dq_metrics)


📊 Computed Data Quality Metrics:
    metric_name  metric_value  \
0  Completeness      1.000000   
1    Uniqueness      1.000000   
2      Accuracy      0.605291   
3    Timeliness      0.115202   
4      Validity      1.000000   

                                               notes  
0        Ratio of non-null fields across all records  
1    Distinct event_id count divided by total events  
2  Average confidence score as proxy for data acc...  
3       Events updated or created within last 1 year  
4                        Share of valid article URLs  


In [8]:
#Insert data into dq_metrics table
# -----------------------------------------------------
dq_metrics.to_sql('dq_metrics', con=engine, if_exists='append', index=False)
print("\n✅ DQ metrics inserted successfully into MySQL!")



✅ DQ metrics inserted successfully into MySQL!


In [9]:
#Verify insertion
# -----------------------------------------------------
check_df = pd.read_sql("SELECT * FROM dq_metrics;", engine)
print("\n✅ Preview of dq_metrics table:")
print(check_df)



✅ Preview of dq_metrics table:
   metric_id   metric_name  metric_value         metric_date  \
0          1  Completeness      1.000000 2025-10-14 12:21:26   
1          2    Uniqueness      1.000000 2025-10-14 12:21:26   
2          3      Accuracy      0.605291 2025-10-14 12:21:26   
3          4    Timeliness      0.115202 2025-10-14 12:21:26   
4          5      Validity      1.000000 2025-10-14 12:21:26   

                                               notes  
0        Ratio of non-null fields across all records  
1    Distinct event_id count divided by total events  
2  Average confidence score as proxy for data acc...  
3       Events updated or created within last 1 year  
4                        Share of valid article URLs  
